In [ ]:
'''Script to extract and sort data of mini ruedi measurments from .txt files for BLANKS'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import plotly.express as px
import os
from datetime import datetime


from pathlib import Path

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [210]:
#sample group directory that should contain all blanks
directory = "/Users/eart0477/Documents/mac_jupyter_files/yellowstone2022/Yellowstone22/raw_data/all_blanks/"

In [211]:
#sort files into categories from a folder. 
txt_files = []
for file in os.listdir(directory):
    if file.endswith(".txt"):
        txt_files.append(os.path.join(file))
        
sample_list = []
standard_list = []
blank_list = []

for string in txt_files:
    if "SAMPLE" in string:
        sample_list.append(string)
    elif "STANDARD" in string:
        standard_list.append(string)
    elif "BLANK" in string:
        blank_list.append(string)


In [212]:
blank_list

['2022-09-06_04-09-03_BLANK.txt',
 '2022-09-16_03-33-09_BLANK.txt',
 '2022-09-15_04-29-55_BLANK.txt',
 '2022-09-17_04-18-42_BLANK.txt',
 '2022-09-13_03-30-47_BLANK.txt',
 '2022-09-21_05-02-31_BLANK.txt',
 '2022-09-19_07-05-44_BLANK.txt',
 '2022-09-05_05-37-04_BLANK.txt',
 '2022-09-20_04-00-59_BLANK.txt',
 '2022-09-08_18-50-36_BLANK.txt']

In [213]:
def processing_function(file_path):   
    
    mz_values = set()

    with open(file_path) as f:



    #extract all mz values        
        for line in f:
            if 'mz=' in line and 'concentration=' not in line:
                mz_start = line.index('mz=') + 3
                space_index = line.index(' ', mz_start)
                mz = line[mz_start:space_index]
                mz_values.add(mz)

    mz_array = list(mz_values)


    #Scan through txt to extract detector and intesity values of every line and store in a nested list grouped by mz

    combined_lists = {mz: {'F': {'PEAK': [], 'ZERO': []}, 'M': {'PEAK': [], 'ZERO': []}} for mz in mz_array}

    with open(file_path) as f:
        for line in f:
            if 'mz=' in line and 'concentration=' not in line:
                mz_start = line.index('mz=') + 3
                space_index = line.index(' ', mz_start)
                mz = line[mz_start:space_index]

                detector_start = line.index('detector=') + 9
                space_index = line.index(' ', detector_start)
                detector = line[detector_start:space_index]

                intensity_start = line.index('intensity=') + 10
                space_index = line.index(' ', intensity_start)
                intensity = line[intensity_start:space_index]

                reading = 'PEAK' if 'PEAK' in line else 'ZERO'

                combined_lists[mz][detector][reading].append(intensity)

    #make a dataframe 

    df = pd.DataFrame(columns=['mz', 'detector', 'reading', 'intensity'])

    for mz in combined_lists:
        for detector in combined_lists[mz]:
            for reading in combined_lists[mz][detector]:
                for intensity in combined_lists[mz][detector][reading]:
                    df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)


    #make sure intensity is numeric                
    df['intensity'] = pd.to_numeric(df['intensity'])

    
     #find sample_name  
    with open(file_path) as f:
        
        # Extract the base filename and extension from the file path
        filename, ext = os.path.splitext(os.path.basename(file_path))

        # Remove spaces from the filename
        file_name = filename.replace(' ', '')
        # find sample_name
        for line in f:
            if 'SAMPLENAME:' in line:
                name_start = line.index('SAMPLENAME:') + 12
                space_index = line.index(' ', name_start)
                sample_name = line[name_start:space_index].strip()
                break
            else:
                sample_name = None 

        # if 'SAMPLENAME:' not found, scan the file again to find 'ANALYSISTYPE:'
        if sample_name is None:
                sample_name = file_name
               
            

       


        return sample_name, df, file_name




In [214]:
def find_mean(sample_name, df):     
    #find mean and std of measurments

        # group the data frame by the 'mz', 'detector' columns
        grouped = df.groupby(['mz', 'detector', 'reading'])

        # calculate the mean and standard deviation of the 'intensity' column for each group
        result = grouped['intensity'].agg(['mean', 'std'])

        # reset the index of the result data frame to make the 'mz' and 'detector' columns columns again
        result = result.reset_index()
        
        peak_F = result[(result['detector'] == 'F') & (result['reading'] == 'PEAK')]



        #Selecting background on M detector
        peak_M = result[(result['detector'] == 'M') & (result['reading'] == 'PEAK')]
        
        final = pd.concat([peak_F, peak_M])
        


        # Initialize an empty dictionary to store the reformatted data
        data_dict = {}

        # Loop through the rows of the original dataframe
        for i, row in final.iterrows():
            # Extract the mz value
            mz = row['mz']

            # Construct column names for mean and std
            mean_col = f"{mz}_F_mean"
            std_col = f"{mz}_F_std"

            # Extract mean and std values
            mean_val = row['mean']
            std_val = row['std']

            # Add the mean and std values to the data_dict
            data_dict[mean_col] = mean_val
            data_dict[std_col] = std_val

        # Create a new dataframe from the data_dict
        new_df = pd.DataFrame([data_dict], index=[sample_name])

      
        
        return final

In [218]:
# Define a list of column names
cols = ['date','sample_name' ,'4_F_mean', '4_F_std', '4_M_mean', '4_M_std', '28_F_mean', '28_F_std', '32_F_mean', '32_F_std',
        '36_M_mean', '36_M_std','40_F_mean', '40_F_std', '40_M_mean', '40_M_std',
        '44_F_mean', '44_F_std',  '84_M_mean', '84_M_std', '14_F_mean', '14_F_std', '15_F_mean', '15_F_std', '16_F_mean', '16_F_std',
        '18_F_mean', '18_F_std',]

# Create an empty dataframe with the specified columns
full_df = pd.DataFrame(columns=cols)

# Iterate through a list of standards
standard_count = 0        
for i in blank_list: 
    standard_count += 1
    path = directory + i
        
    # Run all processing functions for each sample
    sample_name, df,file_name = processing_function(path)
    result = find_mean(sample_name, df)
        
    # Initialize an empty dictionary to store the reformatted data
    data_dict = {}
    
    

    # Loop through the rows of the original dataframe
    for i, row in result.iterrows():
        # Extract the mz value
        mz = row['mz']
        detector = row['detector']

        # Construct column names for mean and std
        mean_col = f"{mz}_{detector}_mean"
        std_col = f"{mz}_{detector}_std"

        # Extract mean and std values
        mean_val = row['mean']
        std_val = row['std']

        # Add the mean and std values to the data_dict
        data_dict[mean_col] = mean_val
        data_dict[std_col] = std_val
        data_dict['sample_name']=sample_name
        
        
    

    # Create a new dataframe from the data_dict
    new_df = pd.DataFrame([data_dict], index=[file_name])

    # Append the new row to the full_df dataframe
    full_df = full_df.append(new_df)




/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is de

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/1452683180.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is de

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/1452683180.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is de

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/1452683180.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_12172/4149104064.py:51: FutureWarning:

The frame.append method is de

In [221]:
for i, index in full_df.iterrows():
    dt_obj = datetime.strptime(i, '%Y-%m-%d_%H-%M-%S_BLANK')
    full_df.loc[i, 'date'] = dt_obj
    full_df.loc[i, 'day'] = dt_obj.day

full_df.sort_values(by='date', inplace=True)


In [223]:
# Save the dataframe
filename = 'all_yellowstone_blanks.xlsx'
full_df.to_excel(filename)


In [222]:
full_df

,date,sample_name,4_F_mean,4_F_std,4_M_mean,4_M_std,28_F_mean,28_F_std,32_F_mean,32_F_std,36_M_mean,36_M_std,40_F_mean,40_F_std,40_M_mean,40_M_std,44_F_mean,44_F_std,84_M_mean,84_M_std,14_F_mean,14_F_std,15_F_mean,15_F_std,16_F_mean,16_F_std,18_F_mean,18_F_std,20_M_mean,20_M_std,20_F_mean,20_F_std,36_F_mean,36_F_std,44_M_mean,44_M_std,day
2022-09-05_05-37-04_BLANK,2022-09-05 05:37:04,2022-09-05_05-37-04_BLANK,7.500000e-15,6.203628e-15,-4.506600e-12,1.254995e-14,5.032000e-14,5.552657e-15,5.914000e-14,4.286374e-15,NaN,NaN,2.120000e-15,6.669108e-15,NaN,NaN,1.205600e-13,5.498454e-15,-4.605720e-12,1.115889e-14,1.162000e-14,5.687442e-15,7.640000e-15,4.867546e-15,6.626000e-14,8.992386e-15,1.673060e-12,1.340518e-13,NaN,NaN,5.440000e-15,5.083601e-15,4.860000e-15,5.133030e-15,7.256639e-10,1.922120e-11,5.0
2022-09-06_04-09-03_BLANK,2022-09-06 04:09:03,2022-09-06_04-09-03_BLANK,-3.630000e-14,8.045806e-15,-1.604380e-12,1.690047e-14,2.752000e-14,9.992347e-15,7.638000e-14,4.599674e-15,6.797050e-12,4.662604e-13,-2.708000e-14,8.707296e-15,6.236900e-13,1.117106e-13,8.906000e-14,7.833135e-15,-1.607640e-12,4.381347e-14,-3.060000e-14,3.761649e-15,-2.322000e-14,1.173784e-14,1.042600e-13,1.442248e-14,4.450300e-12,2.185292e-13,3.314747e-11,1.507076e-12,NaN,NaN,NaN,NaN,NaN,NaN,6.0
2022-09-08_18-50-36_BLANK,2022-09-08 18:50:36,2022-09-08_18-50-36_BLANK,-5.163333e-14,2.050203e-15,-1.649067e-12,1.326483e-14,-9.166667e-15,1.923131e-14,3.260000e-14,2.884441e-15,1.804883e-12,6.203758e-14,-4.640000e-14,5.597321e-15,-8.454333e-13,6.043745e-14,3.140000e-14,9.777014e-15,-1.659917e-12,2.094525e-14,-4.820000e-14,2.622975e-15,-4.373333e-14,7.993956e-15,2.483333e-14,6.951499e-15,2.492767e-12,8.690468e-14,1.731680e-11,6.176277e-13,NaN,NaN,NaN,NaN,NaN,NaN,8.0
2022-09-13_03-30-47_BLANK,2022-09-13 03:30:47,2022-09-13_03-30-47_BLANK,8.200000e-15,7.529276e-15,-1.965060e-12,2.903038e-14,2.796000e-14,7.823235e-15,8.770000e-14,1.556776e-14,-3.545500e-13,1.492944e-13,1.178000e-14,7.842959e-15,-1.728110e-12,3.760617e-14,6.876000e-14,1.162618e-14,-2.100790e-12,9.093363e-15,1.632000e-14,7.224403e-15,1.186000e-14,4.242994e-15,5.670000e-14,1.531388e-14,1.519720e-12,1.214569e-13,1.075460e-11,7.772980e-13,NaN,NaN,NaN,NaN,NaN,NaN,13.0
2022-09-15_04-29-55_BLANK,2022-09-15 04:29:55,2022-09-15_04-29-55_BLANK,-7.124000e-14,3.167491e-15,-2.888840e-12,2.519368e-14,2.754000e-14,1.124380e-14,8.300000e-14,9.021364e-15,7.208300e-13,1.372966e-13,-7.020000e-14,7.395607e-15,-9.098600e-13,1.690398e-13,2.389000e-13,3.356538e-14,-2.965990e-12,3.973537e-14,-7.068000e-14,3.396616e-15,-7.312000e-14,3.153886e-15,1.270800e-13,1.193260e-14,6.774540e-12,2.916737e-13,5.712746e-11,2.474412e-12,NaN,NaN,NaN,NaN,NaN,NaN,15.0
2022-09-16_03-33-09_BLANK,2022-09-16 03:33:09,2022-09-16_03-33-09_BLANK,-7.636000e-14,7.681992e-15,-3.425680e-12,2.443759e-14,-3.758000e-14,3.505282e-15,4.606000e-14,3.367195e-15,-1.331010e-12,1.194732e-13,-7.496000e-14,6.027271e-15,-2.578780e-12,9.609537e-14,6.222000e-14,1.190071e-14,-3.560270e-12,2.176030e-14,-7.654000e-14,5.210374e-15,-7.224000e-14,3.951329e-15,5.034000e-14,1.599150e-14,4.182660e-12,1.956360e-13,3.422771e-11,1.859098e-12,NaN,NaN,NaN,NaN,NaN,NaN,16.0
2022-09-17_04-18-42_BLANK,2022-09-17 04:18:42,2022-09-17_04-18-42_BLANK,4.960000e-15,5.085568e-15,-3.895540e-12,2.699906e-14,1.544000e-14,4.364974e-15,8.114000e-14,1.524264e-14,-3.008430e-12,6.743075e-14,4.600000e-15,3.957272e-15,-3.599560e-12,3.733175e-14,5.404000e-14,6.003582e-15,-3.993950e-12,2.343152e-14,3.140000e-15,5.737857e-15,3.580000e-15,4.120316e-15,4.314000e-14,6.614605e-15,1.430080e-12,1.031763e-13,1.081744e-11,4.965943e-13,NaN,NaN,NaN,NaN,NaN,NaN,17.0
2022-09-19_07-05-44_BLANK,2022-09-19 07:05:44,2022-09-19_07-05-44_BLANK,4.120000e-15,6.154023e-15,-2.854480e-12,1.492622e-14,1.634000e-14,8.186452e-15,9.116000e-14,8.194388e-15,-2.025510e-12,5.676256e-14,2.180000e-15,2.886520e-15,-2.534130e-12,2.827131e-14,5.386000e-14,5.016772e-15,-2.944340e-12,1.036626e-14,5.940000e-15,2.532390e-15,5.700000e-15,6.757588e-15,5.598